In [ ]:
# sagemath version 9.8
# Authors : K.A.Draziotis
# refactoring : M. Adamoudis
# credits : https://latticehacks.cr.yp.to/ntru.html
reset()

In [ ]:
%run ntru-lattice-vfk-cvp.ipynb # code for the CVP on VFK lattices

In [ ]:
%run auxiliary.ipynb

In [ ]:
def keygen():
    while True:
        g = Zx([randomrange3()-1 for i in range(p)])
        if R3(g).is_unit(): break
    grecip = [nicemod3(lift(gri)) for gri in list(1/R3(g))]
    #print(g3,",",nicelift(g3),",",grecip)
    g3 = lst_to_nicepol(grecip) #g3 is 1/R3(g)
    #print(g3,",",nicelift(g3),",",grecip)
    print("is g inverible in R/3?",R3(g3)*R3(g)==1)
    f = randomR()
    h = Rq(g)/(3*Rq(f))
    pk = h
    sk = [Rq(f),g3]
    return pk,sk #X = X mod Rq, pk is in R/q

In [ ]:
def CenterLift(f,q,p):   
    f_balanced = list(   ((f[i]+q//2)%q) -q//2  for i in range(p))
    return Zx(f_balanced)

def encode_ternary(m_x):
    m = m_x.coefficients()
    encoded_m = ''.join(chr((m[i] + 1) % 256) for i in range(len(m)))
    encoded_bytes = encoded_m.encode('utf-8')
    return encoded_bytes

def encode_Rq(f): # f is assumed that has coefficients multiple of 3
    lst = [int(x) for x in (f/3)]
    encoded_f = ''.join(chr((lst[i]) % 256) for i in range(len(lst)))
    encoded_bytes = encoded_f.encode('utf-8')
    return encoded_bytes

def encapsulate(pk,q,p):
    '''
    input  : a public key
    output : a shared secret (key):256 bits and a ciphertext (ct):a polynomial in R/q
    '''
       
    h = pk         # h is in R/q
    r = randomR()  # r is ternary polynomial i.e. coefficients in {-1,0,1}
    hr = h * Rq(r) # this multiplication in in R/q
    m = Zx([-nicemod3(int(hr[i])) for i in range(p)]) # m has coefficients in {-1,0,1}
    ct = Rq(m) + hr  # the coefficients are multiple of 3
    #some checkings
    a = ct-Rq(m)
    b = Rq(h)^(-1)
    print("ct(x) in 3Z[x]? ",[int(i)%3 for i in ct] == len(ct.list())*[0])
    concatenated_string = encode_ternary(r) + encode_Rq(ct)
    shared_key = hashlib.sha512(concatenated_string).hexdigest()[:32]
    return shared_key,ct,m,r

def decap(pk,sk,ct):
    h = pk
    f,g3=sk
    e1 = Rq(3*f) * ct # =3*f*ct
    e2 = [nicelift(e1[i])%3 for i in range(p)]
    e3 = R3(g3) * R3(e2)
    e = Zx([nicemod3(int(e3[i])) for i in range(p)])
    # since we found r we repeat the code from encap
    hr = h * Rq(r) # this multiplication in in R/q
    m = Zx([-nicemod3(int(hr[i])) for i in range(p)]) # m has coefficients in {-1,0,1}
    concatenated_string = encode_ternary(r) + encode_Rq(ct)
    shared_key = hashlib.sha512(concatenated_string).hexdigest()[:32]
    return shared_key

**The Attack**<br>
The first step is to define the matrix 
$$ 
	M_k=
    \left[\begin{array}{c|c}
	I_N & -kI_N  \\
	\hline
	{\textbf 0}_N & qI_N   \\
	\end{array}\right].
    $$
Since it is VFK lattice we have a polynomial algorithm for the CVP, but we need a superbasis.<br>
 The previous matrix does not provide us with a superbasis. We shall need the suitable uniary matrix in 
 order to find it.<br> The following code does also this.

In [ ]:
# step (i)
def init_attack(p,q):
    N=p 
    kappa,P=find_k_and_P(q)
    alpha_vector_vfk = [-kappa] + [0 for i in range(N-1)] 
    A = Zx(alpha_vector_vfk)
    M_k = matrix_for_the_lattice(p,q,A)
    M_NTRU_VFK=unimodular(P,N)*M_k
    # from paper: remark 5.1
    r = - ((kappa*P + kappa - q)*kappa + P + 1)
    s = (kappa*P + kappa - q)*(q - kappa*P ) - (P + 1)*P
    t = (kappa*P-q)*kappa + P
    diag1 = N * ( (1 + P )^2 + ( kappa*(P + 1) - q)^2 )
    diag2 = 1+kappa^2
    #print("r,s,t:",r,s,t)
    basis = M_NTRU_VFK.rows()
    super_basis=get_superbasis(basis)
    Q=get_qij(super_basis)
    print("is VFK?",is_vfk(Q,N))
    return kappa,Q,M_NTRU_VFK,M_k,super_basis



The second step is to define an oracle that provide us with an approximation of the unknown vector. In the code that implements the oracle we also define the target vector. This vector is the connection with our original NTRU problem, in οour case the ntru problem is to find the message (not the private key).
This is modelled as follows.<br>
 The unknown vector is $({\bf m},{\bf u})$ where ${\bf m}\rightarrow m(x)$ and ${\bf u}\rightarrow u(x)=-k(h(x)*r(x)) \pmod{q}$

In [ ]:
def choice_of_E(N,q,kappa,r,h,Range):  
    '''
    Input
    -----
    N,q   : initial parameters of the system
    r     : the nonce polynomial
    h     : the public key polynomial
    ct    : the ciphertext
    Range : a positive integer
    
    output
    ------
    (a list) corresponding to the guessing vector E such that ||E_{i+N} - u_i||<Range
    ''' 
    temp = []
    msg_list = Zx(m).list()    
    hr = h * Rq(r) # this multiplication is in R/q. 
    u  = -kappa*hr # u : the unknown vector u such that u(x)=-kappa(h*r)
    u_list = Zx(u.lift()).list() # convert to list 
    
    #Real_list = msg_list + u_list # the sum here is the concatenation of the two lists 
    u_vector = vector(u_list)     # write u as sage vector
    print
    temp  = u_vector  + vector([randint(-Range,Range) for i in range(N)])
    E = [0]*N + list(temp);
    return E

def target_vector(N,E,kappa,ct):
    '''
    Input
    -----
    N   : initial param of NTRU-prime, N=p
    E   : the guessing vector, from the oracle
    ct  : the ciphertext
    
    Output
    ------
    the target vector of the form
    (0_N,b1+E1,...,bN+EN)
    '''  
    b1=kappa*ct # the polynomial k*ct(x) in R/q
    b=Zx(b1.lift())
    Blist=b.coefficients(sparse=False)
    if len(Blist)==N:
        t = vector(N*[0] + Blist) + vector(E)
    return t # the target vector t = (0_N,b) + E    
    


In [ ]:
#parameters for NTRU prime KEM
# returns p,q,w
def sntrup(x):
    if x==1:
        return 653,4621,288
    if x==2:
        return 761,4591,286
    if x==3:
        return 857,5167,322
    if x==4:
        return 953,6343,396
    if x==5:
        return 1013,7177,448
    if x==6:
        return 1277,7879,492
    

In [ ]:
p,q,w=sntrup(1)
t=w/2
#p,q,w=7,103,6 # w even
#t=w/2
Zx.<x> = ZZ[]; 
R.<xp> = Zx.quotient(x^p-x-1)
Fq = GF(q); 
Fqx.<xq> = Fq[]; 
Rq.<X> = Fqx.quotient(x^p-x-1)
F3 = GF(3); F3x.<x3> = F3[]; R3.<x3p> = F3x.quotient(x^p-x-1)
pk,sk=keygen()
h = pk
N = p

In [ ]:
# intensive part  [only one time]
# Computation of superbasis and Selling parameter in a large matrix
import time
start=time.time()
kappa,Q,M_NTRU_VFK,M_k,superbasis = init_attack(p,q)
M_NTRU,M_NTRU_fplll    = LLL_reduction_of_M_NTRU(M_NTRU_VFK)
print("time:",time.time()-start)

In [ ]:
# new encapsulation
def new_encap(pk,q,p):
    ses_key,ct,m,r=encapsulate(pk,q,p)
    m_list=correction_of_msg(N,m)
    rq = (ct-Rq(m))*(Rq(h)^(-1))
    #print("N=",N)
    #print("m=",m,", coeff=",m.list())
    #print("ses_key=",ses_key)
    #print("decap=",decap(pk,sk,ct)==ses_key)
    return m,m_list,r,ct



In [ ]:
# Babai
m,m_list,r,ct = new_encap(pk,q,p)
for i in range(100):
    Range = 51
    E  = choice_of_E(N,q,kappa,r,h,Range)
    target = target_vector(N,E,kappa,ct)
    M_GSO = GSO.Mat(M_NTRU_fplll)
    M_GSO.update_gso()
    L_babai = M_GSO.babai(target)
    w_babai = sum(-L_babai[i]*M_NTRU[i] for i in range(M_NTRU_fplll.nrows)).list() # why minus?
    print("Success?",w_babai[0:N]==m_list)
    print("distance:",(vector(target)+vector(w_babai)).norm().n())
    hits(w_babai[0:N],m_list)
    print("\n")

In [ ]:
### using CVP oracle
#M_NTRU_VFK=unimodular(P,N)*init_M_NTRU
#Reminder : N,Q,basis,basis_superbasis remain the same for every instance
m,m_list,r,ct = new_encap(pk,q,p)
for i in range(100):
    Range = 51
    E  = choice_of_E(N,q,kappa,r,h,Range)
    target = target_vector(N,E,kappa,ct)
    start=time.time()
    basis = M_NTRU_VFK.rows()
    L_vfk=cvp_vfk(N,Q,basis,superbasis,target)
    w_vfk =[-x for x in L_vfk]
    print("CVP done")
#w_vfk = sum(-L_vfk[i]*M_NTRU[i] for i in range(M_NTRU_fplll.nrows)).list() # why minus?
    print(w_vfk[0:N]==m_list)
    print("time for CVP:",time.time()-start)
    hits(w_vfk[0:N],m_list)
    print("\n")